In [ ]:
# Model the usage of MTB trails as a function of weather conditions
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import utils
import math
from sklearn.linear_model import LinearRegression, Ridge
import numpy as np
from datetime import date, timedelta
import random

In [ ]:
import warnings
# do we really need this? is it version-dependent?
warnings.filterwarnings('ignore')

In [ ]:
# gather data
md = utils.get_segment_metadata()
# ignore inactive segments
md = md[md['active_modeling']]

#md['closest_ims'] = md['closest_ims'].astype(int)

In [ ]:
rl_ = utils.get_ridelogs()

# Trim junk
md = md[['id', 'name', 'closest_ims']].copy()

In [ ]:
d5 = rl_.copy()

In [ ]:
# add the closest IMS station
d6 = d5.merge(md[['id', 'closest_ims', 'name']], how='right', left_on=['segment_id'], right_on=['id'])
#md[['name','closest_ims']]

In [ ]:
weather_days = utils.get_weather_days(d6)

In [ ]:
# Add rain measurements
data = d6.merge(weather_days, how='left', left_on=['closest_ims', 'date'], right_on=['closest_ims', 'date'])

# cumulative measures of rainfall

data.sort_values('date', inplace=True)

In [ ]:
# add lockdown value
data['lockdown'] = 0
data.loc[data['date'].between('2021-01-07', '2021-02-06'), 'lockdown'] = 1

In [ ]:
data['rain_7d'] = data.fillna(0).groupby('segment_id')['rain_mm'].apply(lambda x : x.rolling(7).sum().clip(lower=0))
#data['soil_moisture'] = data.groupby('segment_id')['rain_mm'].apply(utils.bathtub)
df_orig = data.copy()

In [ ]:
u_w = 0.1   # how much weight to give to the uniformity measure

def bathtub_set(data_, soilmodel, include_rain=True, **kwargs):
    # we get no information from days with dry soil - there's many of them and nothing to learn from
    # filter out days with no rain in last 7 days
    #print(data_.shape)
    rows_included = data_['rain_7d'].fillna(-1).values > 0
    data_ = data_.loc[rows_included]
    #print(data_.shape)
    soil = np.array(soilmodel(data_[['rain_mm', 'wind_ms']], **kwargs)).reshape(-1, 1)
    if include_rain:
        rain = (np.array(data_['rain_morning']) > 5).astype(int)    # indicator for: was there any rain on this particular morning?   
    else:
        rain = np.zeros_like(data_['rain_mm'])                 # fill with zeros
    rain = rain.reshape(-1, 1)
    temp = (np.array(data_['temp_morning']) > 12).astype(int)    # indicator for: was it a warm morning?
    temp = temp.reshape(-1, 1)
    lockdown = np.array(data_['lockdown']).reshape(-1, 1)
    X = np.concatenate((soil, rain, lockdown, temp), axis=1)
    y = data_['nrides']
    cmap = ['soil', 'rain', 'lockdown', 'temp']
    return (X, y, cmap)

def uniformity(vv):
    # measure how a univariate distribution is uniform, on a scale of 0 (one-hot) to 1 (perfectly uniform)
    # from https://stats.stackexchange.com/questions/25827/how-does-one-measure-the-non-uniformity-of-a-distribution
    d=3    # number of bins
    hist, bin_edges = np.histogram(vv, bins=d, density=False)
    hist = hist/np.sum(hist)
    l2 = np.linalg.norm(hist)
    u = (l2*np.sqrt(d)-1) / (np.sqrt(d)-1) 
    return 1-u

def regress(X, y, sanity_checks=True):
    # remove NaNs. We do this by stuffing everything into a dataframe first
    dfXy = pd.DataFrame(X)
    dfXy['y'] = y.values
    dfXy.dropna(inplace=True)
    # now unpack
    Xy = np.array(dfXy.values)
    X = Xy[:,:-1]
    y = Xy[:, -1].reshape(-1, 1)

    nrows = X.shape[0]
    # skip if there's too little data
    # skip if the moisture model didn't give us examples of dry soil (below 1)
    # skip if the moisture model didn't give us examples of non-dry soil (above 1)
    # skip if there isn't enough variation in the Y values
    if (nrows <= 2) or (X[:, 0].min() > 1) or (X[:, 0].max() <= 1) or (y.ptp() < .25):
        return {'coef' : None, 'intercept' : None, 'score' : -1}
    # We typically get many many examples with X=0 (dry soil), down-weigh them to at most 1/3 of all data
    Xsoil = X[:, 0].reshape(-1)
    u_val = uniformity(Xsoil)
    weights = np.ones(Xsoil.shape)
    nzeros = np.count_nonzero(Xsoil == 0)
    if nzeros > 0:
        norm_factor = min(1.0, (1/3.0)*(1.0*nrows/nzeros))
        if False and random.random() < 0.001:
            print("%d rows %d nzeros factor %f" % (nrows, nzeros, norm_factor))
        # we collapse all the zero points to have a total weight of one
        weights[Xsoil == 0] = norm_factor
    try:
        reg = Ridge(alpha=1E-6, normalize=True).fit(X, y, sample_weight = weights)
        #reg = LinearRegression(normalize=True).fit(X, y, sample_weight = weights)
        coef = reg.coef_[0]
        score = reg.score(X, y, sample_weight = weights)
        # apply some sanity checks
        if sanity_checks:
            # we need more soil moisture = less rides, not the opposite
            if coef[0] > 0:
                score = -1
            # we need more rain today = less rides, not the opposite
            if coef[1] > 0:
                score = -1
            # we need more lockdown today = less rides, not the opposite
            if coef[2] > 0:
                score = -1
        return {'coef' : reg.coef_[0], 'intercept' : reg.intercept_[0], 'score' : score, 'uniformity' : u_val}
    except ValueError:   # probably not enough data
        return {'coef' : None, 'intercept' : None, 'score' : -1, 'uniformity' : -1}

def best_bathtub(data_):
    mydata = data_.copy()
    out = []

    # Try the geometric model
    clist = list(np.arange(4, 80, 4))
    #clist = list(np.arange(4, 50, 8))

    dlist = list(np.arange(0.5, 0.65, 0.05))
    dlist.extend(np.arange(0.65, 0.8, 0.05))
    dlist.extend(np.arange(0.8, 1, 0.05))

    wlist = list(np.arange(0, 3, 0.25))
    #wlist = [0, 1]
    #wlist = []

    for c in clist:
        for d in dlist:
            for w in wlist:
                X, y, cmap = bathtub_set(mydata, utils.bathtub_geom_, capacity=c, drainage_factor=d, fwind=w)
                p = regress(X, y)
                if p['score'] > -1:
                    outdict = {'f': 'bathtub_geom', 'capacity': c, 'drainage_factor' : d, 'fwind' : w,
                               'intercept' : p['intercept']}
                    for ci in range(len(cmap)):
                        outdict['c_' + cmap[ci]] = p['coef'][ci]
                    out.append([p['score'], p['uniformity'], outdict])
    
    #Try the basic model
    clist = list(np.arange(1, 10, 0.5))
    clist.extend(range(10,80,4))
    #clist = range(10, 30, 5)
    
    dlist = list(np.arange(5, 10, 0.5))
    dlist.extend(range(10,25))
    
    wlist = list(np.arange(0, 3, 0.25))
    #wlist = []
    
    for c in clist:
        for d in dlist:
            for w in wlist:
                # we want to ensure some margin between capacity and drainage
                if c > d + 2:
                    X, y, cmap = bathtub_set(mydata, utils.bathtub_, capacity=c, drainage=d, fwind=w)
                    p = regress(X, y)
                    if p['score'] > -1:
                        outdict = {'f': 'bathtub', 'capacity': c, 'drainage' : d, 'fwind' : w,
                                             'intercept' : p['intercept']}
                        for ci in range(len(cmap)):
                            outdict['c_' + cmap[ci]] = p['coef'][ci]
                        out.append([p['score'], p['uniformity'], outdict])

    #Try the day-counter model
    tlist = [1, 5, 7.5, 10, 15]
    
    dlist = range(1, 7)
    
    wlist = list(np.arange(0, 2, 0.2))
    #wlist = []
    
    for t in tlist:
        for d in dlist:
            for w in wlist:
                    X, y, cmap = bathtub_set(mydata, utils.daycounter_, include_rain=False, cday=d, rain_thresh=t, fwind=w)
                    p = regress(X, y)
                    if p['score'] > -1:
                        outdict = {'f': 'daycounter', 'cday' : d, 'rain_thresh': t, 'fwind' : w,
                                             'intercept' : p['intercept']}
                        for ci in range(len(cmap)):
                            outdict['c_' + cmap[ci]] = p['coef'][ci]
                        out.append([p['score'], p['uniformity'], outdict])

                        
    if(len(out) > 0):
        cdf = pd.DataFrame(out, columns=['score', 'uniformity', 'par'])
        cdf['cscore'] = (1.0-u_w)*cdf['score'] + u_w*cdf['uniformity']
        idxmax = cdf['cscore'].idxmax()
        if(math.isnan(idxmax)):
            idxmax = 0
        return cdf.iloc[idxmax]
    else:
        return None

In [ ]:
df = df_orig.copy()
out = []
d = {}
seglist = df['segment_id'].unique()
#seglist = ['4136318']
for seg in seglist:
    print(seg, end="...")
    mydata = df.query("segment_id == @seg")
    res = best_bathtub(mydata)
    if res is not None:
        res = res.to_dict()
        res['segment_id'] = seg
        out.append(res)
print("")

In [ ]:
params = pd.DataFrame(out)
# Compute the days to dry
params['dtd'] = None

exploded = pd.DataFrame.from_records(params['par'])
params = pd.concat([params, exploded], axis='columns')

# check we have all needed columns
for c in ['drainage', 'drainage_factor', 'rain_thresh', 'capacity', 'cday']:
    if c not in params.columns:
        params[c] = None


# We compute the moisture value which yields 0.9 of the intercept (soil is 90% dry)
params['y90'] = 0.9*exploded['intercept']
params['x90'] = (params['y90'] - exploded['intercept'])/exploded['c_soil']

# compute for the additive model
rows = (params['f'] == 'bathtub')
params.loc[rows, 'dtd'] = (params['capacity'] - params['x90'])/params['drainage']

# compute for the geometric model
rows = (params['f'] == 'bathtub_geom')
# how many times to multiply by the factor until we reach a value of x90?
params.loc[rows, 'dtd'] = params[rows].apply(lambda r: (math.log(r['y90']) -math.log(r['capacity']))/math.log(r['drainage_factor']), axis=1)

# compute for the daycounter model
rows = (params['f'] == 'daycounter')
params.loc[rows, 'dtd'] = params.loc[rows, 'cday']

params = params.merge(md, how='left', left_on='segment_id', right_on='id').drop(columns='id')
params[['segment_id', 'score', 'par']].to_csv('data/segments/params.csv', float_format='%.3g', index=False)

params[['name', 'dtd', 'score', 'uniformity', 'capacity', 'drainage', 'drainage_factor', 'cday', 'rain_thresh', 'fwind', 'c_lockdown', 'c_rain', 'c_soil', 'c_temp']].sort_values('score')

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8,8))
sns.set_style('ticks')

exploded = pd.DataFrame.from_records(params['par'])
exploded['score'] = params['score']

# plot for the additive model
rows = (exploded['f'] == 'bathtub')
if rows.sum() > 0:
    sns.scatterplot(data=exploded[rows], x='capacity', y='drainage', size='score', hue='fwind', ax=ax[0])
    ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
# plot for the geometric model
rows = (exploded['f'] == 'bathtub_geom')
if rows.sum() > 0:
    sns.scatterplot(data=exploded[rows], x='capacity', y='drainage_factor', size='score', hue='fwind', ax=ax[1])
    ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
# plot for the daycounter model
rows = (exploded['f'] == 'daycounter')
if rows.sum() > 0:
    sns.scatterplot(data=exploded[rows], x='cday', y='rain_thresh', size='score', hue='fwind', ax=ax[2])
    ax[2].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

fig.tight_layout(pad=1)

In [ ]:
names = df['segment_id'].unique()
fig, ax = plt.subplots(figsize=(10,50), nrows=len(names), ncols=1)

for vi in range(len(names)):
    seg = names[vi]
    mydata = df.query("segment_id == @seg").copy()
    p = params.query("segment_id == @seg")
    if len(p) > 0:
        f = p.iloc[0].par['f']
        name = mydata.iloc[0]['name'][::-1]
        score = p.iloc[0]['score']

        fwind = p.iloc[0].par['fwind']

        if f == 'bathtub':
            capacity = p.iloc[0].par['capacity']
            drainage = p.iloc[0].par['drainage']
            par_str = f'c=%g d=%g w=%g' % (capacity, drainage, fwind)
            X, y, cmap = bathtub_set(mydata, utils.bathtub_, capacity=capacity,
                                     drainage=drainage, fwind=fwind)
        elif f == 'bathtub_geom':
            capacity = p.iloc[0].par['capacity']
            drainage_factor = p.iloc[0].par['drainage_factor']
            fwind = p.iloc[0].par['fwind']
            par_str = f'c=%g d_f=%g w=%g' % (capacity, drainage_factor, fwind)
            X, y, cmap = bathtub_set(mydata, utils.bathtub_geom_, capacity=capacity,
                                     drainage_factor=drainage_factor, fwind=fwind)
        elif f == 'daycounter':
            cday = p.iloc[0].par['cday']
            rain_thresh = p.iloc[0].par['rain_thresh']
            fwind = p.iloc[0].par['fwind']
            par_str = f'cday=%g r_t=%g w=%g' % (cday, rain_thresh, fwind)
            X, y, cmap = bathtub_set(mydata, utils.daycounter_, cday=cday,
                                     rain_thresh=rain_thresh, fwind=fwind)

        else:
            print("Uh")
        plot_data = pd.DataFrame(data = {'soil_moisture' : X[:, 0], 'nrides' : y})
        sns.scatterplot(data=plot_data,
                        y='nrides', x='soil_moisture',
                        marker='o',
                        ax=ax[vi]).set_title(f'%s score=%.2f %s' % (name, score, par_str))

fig.tight_layout(pad=1)

In [ ]:
if False:
    data=by_dow.reset_index()
    #sns.barplot(data=data, hue='segment_id', x='rides_dow', y='weekday', orient='h')
    #data.plot.barh()
    data = data.pivot_table(index='weekday', columns='segment_id', values='rides_dow').apply(lambda x: x*100/sum(x), axis=0)
    data.T.plot(kind="bar", stacked=True)
    data